In [162]:
#import libraries to read the data
import pandas as pd
import numpy as np
from scipy.stats import mode
#reading training and testing file
train = pd.read_csv('train_loan.csv')
test = pd.read_csv('test_loan.csv')
#function for missing values
def num_missing(x):
  return sum(x.isnull())
#Cleaning training data
print (train.apply(num_missing, axis=0))
train = train.replace('[+]','',regex=True)
train['Gender'].fillna(mode(train['Gender'].astype('str')).mode[0], inplace=True)
train['Married'].fillna(mode(train['Married'].astype('str')).mode[0], inplace=True)
train['Self_Employed'].fillna(mode(train['Self_Employed'].astype('str')).mode[0], inplace=True)
train['Dependents'] = train['Dependents'].astype(float)
#train['Loan_Amount_Term'].fillna(mode(train['Loan_Amount_Term']).mode[0], inplace=True)
#train['Credit_History'].fillna(mode(train['Credit_History']).mode[0], inplace=True)
#train['Dependents'].fillna(mode(train['Dependents'].astype('float')).mode[0], inplace=True)
#from fancyimpute import KNN    
# X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN
# Use 3 nearest rows which have a feature to fill in each row's missing features
#X_filled_knn = KNN(k=3).complete(train)

# for imputing loan amount 
#impute_grps = train.pivot_table(values=["LoanAmount"], index=["Gender","Married","Self_Employed"], aggfunc=np.mean)
#for i,row in train.loc[train['LoanAmount'].isnull(),:].iterrows():
#  ind = tuple([row['Gender'],row['Married'],row['Self_Employed']])
#  train.loc[i,'LoanAmount'] = impute_grps.loc[ind].values[0]
#for one hot coding
train = pd.get_dummies(train,columns=['Loan_Status'],drop_first=True,sparse=False)
train.rename(index=str, columns={"Loan_Status_Y":"Loan_Status"},inplace=True)
train_dummies = pd.get_dummies(train, columns=[u'Gender',
       u'Married', u'Education', u'Self_Employed', u'Property_Area'],drop_first=False,sparse=False)

#cleaning test data 

test = test.replace('[+]','',regex=True)
test['Gender'].fillna(mode(test['Gender'].astype('str')).mode[0], inplace=True)
test['Married'].fillna(mode(test['Married'].astype('str')).mode[0], inplace=True)
test['Self_Employed'].fillna(mode(test['Self_Employed'].astype('str')).mode[0], inplace=True)
test['Dependents'] = test['Dependents'].astype(float)
#test['Loan_Amount_Term'].fillna(mode(test['Loan_Amount_Term']).mode[0], inplace=True)
#test['Credit_History'].fillna(mode(test['Credit_History']).mode[0], inplace=True)
#test['Dependents'].fillna(mode(test['Dependents'].astype('float')).mode[0], inplace=True)
# for imputing loan amount 
#impute_grps = test.pivot_table(values=["LoanAmount"], index=["Gender","Married","Self_Employed"], aggfunc=np.mean)
#for i,row in test.loc[test['LoanAmount'].isnull(),:].iterrows():
#  ind = tuple([row['Gender'],row['Married'],row['Self_Employed']])
#  test.loc[i,'LoanAmount'] = impute_grps.loc[ind].values[0]

#test = pd.get_dummies(test,columns=['Loan_Status'],drop_first=True,sparse=False)
#test.rename(index=str, columns={"Loan_Status_Y":"Loan_Status"},inplace=True)
#for one hot coding
test_dummies = pd.get_dummies(test, columns=[u'Gender',
       u'Married', u'Education', u'Self_Employed', u'Property_Area'],drop_first=False,sparse=False)
print (train.apply(num_missing, axis=0))
# removing loan_id and status from training data
X_train = train_dummies.drop(['Loan_ID'], 1)
X_train = X_train.drop(['Loan_Status'], 1)                        
Y_train = train_dummies['Loan_Status']
#removing loan id from testing data
X_test = test_dummies.drop(['Loan_ID'],1)




Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64
Loan_ID               0
Gender                0
Married               0
Dependents           15
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


C:\Users\rahul.r.rajaram\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [177]:
#Import XGboost classifier
from xgboost.sklearn import XGBClassifier


#Create a Gaussian Classifier
model = XGBClassifier(
 learning_rate =0.12,
 n_estimators=1000,
 max_depth=8,
 min_child_weight=3,
 gamma=0.35,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 reg_alpha = 1,
 seed=27)

#model training
model.fit(X_train,Y_train)
y_pred = model.predict(X_test)


C:\Users\rahul.r.rajaram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


C:\Users\rahul.r.rajaram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Credit_History             0.270463
ApplicantIncome            0.236814
LoanAmount                 0.197971
CoapplicantIncome          0.126236
Dependents                 0.052834
Loan_Amount_Term           0.046885
Property_Area_Semiurban    0.023920
Education_Not Graduate     0.022449
Property_Area_Rural        0.022428
dtype: float64

[1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1
 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1
 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1]


In [178]:
'''Getting SVM Predictions '''
df_predicted = pd.DataFrame(y_pred,columns=["Loan_Status"])
result_xgb = pd.concat([test['Loan_ID'],df_predicted],axis=1)
result_xgb['Loan_Status'] = result_xgb['Loan_Status'].map({1:'Y', 0: 'N'})
result_xgb.to_csv("result_xgb_final.csv", index = False)
